In [2]:
from transformers import GPT2Model, GPT2Tokenizer
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm

In [32]:
df = pd.read_csv('C:/Users/farez/amazon_reviews.csv')
del df[df.columns[0]] # first column is just index so we drop

In [30]:
print(df)

      reviewerName  overall  \
0              NaN      4.0   
1             0mie      5.0   
2              1K3      4.0   
3              1m2      5.0   
4     2&amp;1/2Men      5.0   
...            ...      ...   
4910        ZM "J"      1.0   
4911            Zo      5.0   
4912     Z S Liske      5.0   
4913      Z Taylor      5.0   
4914           Zza      5.0   

                                             reviewText  reviewTime  day_diff  \
0                                            No issues.  2014-07-23       138   
1     Purchased this for my device, it worked as adv...  2013-10-25       409   
2     it works as expected. I should have sprung for...  2012-12-23       715   
3     This think has worked out great.Had a diff. br...  2013-11-21       382   
4     Bought it with Retail Packaging, arrived legit...  2013-07-13       513   
...                                                 ...         ...       ...   
4910  I bought this Sandisk 16GB Class 10 to use wit...  201

In [41]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data_list):
        self.data_list = data_list

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        return self.data_list[idx]

text = df['reviewText'].tolist()
text = [item for item in text if isinstance(item, str)] # removing any reviews that are not strings
dataset = MyDataset(text)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_text = [train_dataset[i] for i in range(len(train_dataset))]
test_text = [test_dataset[i] for i in range(len(test_dataset))]

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

tokenizer.pad_token = tokenizer.eos_token  # Use the end of sequence token as padding token.
encoded_inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")


embeddings = []
# obtain embeddings

chunk = 10
for i in tqdm(range(0,len(encoded_inputs['input_ids']),chunk)):
    with torch.no_grad():
        outputs = model(encoded_inputs['input_ids'][i:i+chunk])
        hidden_states = outputs.last_hidden_state
        embeddings.append(torch.mean(hidden_states, dim=1))  # mean pooling

print(embeddings)

  0%|          | 0/492 [00:00<?, ?it/s]

Chunk 1: [1, 2, 3, 4]
Chunk 2: [5, 6, 7, 8]
Chunk 3: [9]


In [47]:
len(encoded_inputs['input_ids'])

3931